# Train an XGBoost for tracing lineage


In [1]:
import bread
import os
import pandas as pd
import numpy as np
path_to_data = os.path.abspath('data')
path_to_colonies = os.path.abspath('data/colonies')


In [2]:
# load the 5 colony ground truth
colonies_gt = pd.DataFrame()
for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_lineage.csv'.format(i)))
    temp_colony['colony'] = i
    colonies_gt = colonies_gt.append(temp_colony)
colonies_gt = colonies_gt.reset_index(drop=True)
colonies_gt.rename(columns={'# parent_id': 'parent_GT'}, inplace=True)
colonies_gt


AttributeError: 'DataFrame' object has no attribute 'append'

In [ ]:
from bread.algo.lineage import LineageGuesserML
from bread.data import Segmentation, Lineage, SegmentationFile


def extract_features(segmentation_path, args):
    candidate_features = pd.DataFrame(columns=['bud_id', 'candid_id', 'time_id', 'feature1', 'feature2',
                                      'feature3', 'feature4', 'feature5', 'feature6', 'feature7', 'feature8', 'feature9', 'feature10'])
    segmentation = SegmentationFile.from_h5(
        segmentation_path).get_segmentation('FOV0')
    guesser = LineageGuesserML(
        segmentation=segmentation,
        nn_threshold=args["nn_threshold"],
        flexible_nn_threshold=args["flexible_nn_threshold"],
        num_frames_refractory=args["num_frames_refractory"],
        num_frames=args["num_frames"],
        bud_distance_max=args["bud_distance_max"]
    )
    bud_ids, time_ids = segmentation.find_buds(
    ).bud_ids, segmentation.find_buds().time_ids
    for i, (bud_id, time_id) in enumerate(zip(bud_ids, time_ids)):
        frame_range = guesser.segmentation.request_frame_range(
            time_id, time_id + guesser.num_frames)
        num_frames_available = guesser.num_frames
        if len(frame_range) < 2:
            # raise NotEnoughFramesException(bud_id, time_id, guesser.num_frames, len(frame_range))
            print("Not enough frames for bud {} at time {}. Only {} frames available.".format(
                bud_id, time_id, len(frame_range)))
        if len(frame_range) < guesser.num_frames:
            num_frames_available = len(frame_range)
            # warnings.warn(NotEnoughFramesWarning(bud_id, time_id, guesser.num_frames, len(frame_range)))
            print("Not enough frames for bud {} at time {}. Only {} frames available.".format(
                bud_id, time_id, len(frame_range)))
        # check the bud still exists !
        for time_id_ in frame_range:
            if bud_id not in guesser.segmentation.cell_ids(time_id_):
                # raise LineageGuesserExpansionSpeed.BudVanishedException(bud_id, time_id_)
                print("Bud {} vanished at time {}".format(bud_id, time_id_))
        selected_times = [i for i in range(
            time_id, time_id + num_frames_available)]
        candidate_parents = guesser._candidate_parents(
            time_id, nearest_neighbours_of=bud_id)
        summary_features = np.zeros(
            (len(candidate_parents), guesser.number_of_features), dtype=np.float64)
        for c_id, candidate in enumerate(candidate_parents):
            try:
                features, _ = guesser._get_ml_features(
                    bud_id, candidate, time_id, selected_times)
                new_row = {'bud_id': bud_id, 'candid_id': candidate, 'time_id': time_id, 'feature1': features[0], 'feature2': features[1], 'feature3': features[2], 'feature4': features[
                    3], 'feature5': features[4], 'feature6': features[5], 'feature7': features[6], 'feature8': features[7], 'feature9': features[8], 'feature10': features[9]}
                candidate_features = candidate_features.append(
                    new_row, ignore_index=True)
            except:
                print("Error for bud {} at time {} with candidate {}".format(
                    bud_id, time_id, candidate))
    return candidate_features


In [ ]:
# extract features for each colony and save them to a csv file. This takes a while. But after this, we only need to load the csv files. every time.
colony_candidate_features = [0, 0, 0, 0, 0, 0]
args = {"nn_threshold": 8.0, "flexible_nn_threshold": True,
        "num_frames_refractory": 0, "num_frames": 4, "bud_distance_max": 10}
for i in [1, 2, 3, 4, 5]:
    print("finding features for colony {}".format(i))
    colony_candidate_features[i] = extract_features(os.path.join(
        path_to_colonies, 'colony00{}_segmentation.h5'.format(i)), args)
    colony_candidate_features[i]['colony'] = i
    # remove rows with time_id = 0
    colony_candidate_features[i] = colony_candidate_features[i][colony_candidate_features[i].time_id != 0]
    # remove rows with bud_id == candid_id (this shouldn't happen in theory)
    colony_candidate_features[i] = colony_candidate_features[i][colony_candidate_features[i].bud_id !=
                                                                colony_candidate_features[i].candid_id]
    # save to csv
    colony_candidate_features[i].to_csv(os.path.join(
        path_to_colonies, 'colony00{}_candidate_features.csv'.format(i)), index=False)


In [ ]:
colonies_features = pd.DataFrame()

for i in [1, 2, 3, 4, 5]:
    temp_colony = pd.read_csv(os.path.join(
        path_to_colonies, 'colony00{}_candidate_features.csv'.format(i)))
    colonies_features = colonies_features.append(temp_colony)
colonies_features = colonies_features.reset_index(drop=True)
colonies_features


In [ ]:
# helper functions for training XGBoost
import numpy as np
import itertools
import xgboost as xgb


In [ ]:
def get_matrix_features(features_all, lineage_gt):
    # Generate np array of feature sets for each bud
    df1 = lineage_gt.copy()
    # remove the rows with parent_GT = -1 (no parent) and the rows with candid_GT = -2 (disappearing buds)
    df1 = df1.loc[df1.parent_GT != -1]
    df1 = df1.loc[df1.parent_GT != -2]
    df2 = features_all.copy()

    features_list = []
    parent_index_list = []
    candidate_list = []
    for bud, colony in df1[['bud_id', 'colony']].values:
        bud_data = df2.loc[(df2['bud_id'] == bud) & (df2['colony'] == colony)]
        candidates = bud_data['candid_id'].to_numpy()
        if candidates.shape[0] < 4:
            candidates = np.pad(
                candidates, ((0, 4 - candidates.shape[0])), mode='constant', constant_values=-3)
        features = bud_data[['feature1', 'feature2', 'feature3', 'feature4', 'feature5',
                             'feature6', 'feature7', 'feature8', 'feature9', 'feature10']].to_numpy()
        if features.shape[0] < 4:
            features = np.pad(features, ((
                0, 4 - features.shape[0]), (0, 0)), mode='constant', constant_values=-1)
        if features.shape[0] > 4:
            sorted_indices = np.argsort(features[:, 0])
            print('more than 4 candidates', bud, colony, candidates, sorted_indices, int(df1.loc[(df1['bud_id'] == bud) & (
                df1['colony'] == colony), 'parent_GT']))
            # slice the top 4 rows
            k = 4
            features = features[sorted_indices[:k]]
            candidates = candidates[sorted_indices[:k]]

        parent = int(df1.loc[(df1['bud_id'] == bud) & (
            df1['colony'] == colony), 'parent_GT'])
        # print(bud, colony, parent)
        # print(candidates)
        if(parent not in candidates):
            print('parent not in candidates', bud, colony, candidates, parent)
            # remove this from the df
            df1.drop(df1.loc[(df1['bud_id'] == bud) & (df1.colony == colony)].index,
                     inplace=True)
            continue
        else:
            parent_index = np.where(candidates == parent)[0][0]

        parent_index_list.append(parent_index)
        features_list.append(features)
        candidate_list.append(candidates)
    df1['features'] = features_list
    df1['candidates'] = candidate_list
    df1['parent_index_in_candidates'] = parent_index_list
    return df1


In [ ]:
colonies_matrix_features = get_matrix_features(colonies_features, colonies_gt)


In [ ]:
# functions for plotting
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


def plot_feature_importance(bst, figsize=(10, 5), title=''):
    """
    Plots the feature importances of an XGBoost model
    """
    plt.figure(figsize=figsize)
    feature_importance = bst.get_score(importance_type='weight')
    feature_importance = {k: v for k, v in sorted(
        feature_importance.items(), key=lambda item: item[1], reverse=True)}
    plt.bar(range(len(feature_importance)), list(
        feature_importance.values()), align='center')
    plt.xticks(range(len(feature_importance)), list(
        feature_importance.keys()), rotation=90)
    plt.xlabel('Features')
    plt.ylabel('Feature importance score')
    plt.title('Feature importance for ' + title)
    plt.show()


def plot_eval_metrics(evals_result, title=''):
    """
    Plot evaluation metrics from an XGBoost model
    """
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)

    try:
        plt.plot(evals_result['train']['merror'], label='Train')
        plt.plot(evals_result['test']['merror'], label='Validation')

    except:  # For binary classification, the loss is 'error'
        plt.plot(evals_result['train']['error'], label='Train')
        plt.plot(evals_result['test']['error'], label='Validation')

    plt.xlabel('Number of Boosting Rounds')
    plt.ylabel('Error')
    plt.title('Error vs Number of Boosting Rounds for ' + title)
    plt.legend()


def plot_confusion_matrix(y_true, y_pred):
    plt.figure()
    plt.imshow(confusion_matrix(y_true, y_pred), cmap='Blues')
    plt.title('Confusion matrix')
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


def print_matrix(s):
    # Do heading
    print("     ", end="")
    for j in range(len(s[0])):
        print("%5d " % j, end="")
    print()
    print("     ", end="")
    for j in range(len(s[0])):
        print("------", end="")
    print()
    # Matrix contents
    for i in range(len(s)):
        print("%3d |" % (i), end="")  # Row nums
        for j in range(len(s[0])):
            print("%.3f " % (s[i][j]), end="")
        print()


In [ ]:
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


def train_xgboost_matrix(matrix_features_df, augment=True):
    X = matrix_features_df['features'].to_numpy()
    y = matrix_features_df['parent_index_in_candidates'].to_numpy()

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, shuffle=True)
    if augment:
        X_train, y_train = generate_all_permutations(X_train, y_train)
    X_train = flatten_3d_array(X_train)
    X_test = flatten_3d_array(X_test)

    # Train the default XGBoost model
    dtrain = xgb.DMatrix(X_train, label=y_train)
    params = {'objective': 'multi:softmax',
              'num_class': 4, 'eval_metric': 'merror'}
    model = xgb.train(params, dtrain, num_boost_round=100)
    # Make predictions and evaluate the model
    dtest = xgb.DMatrix(X_test, label=y_test)
    preds = model.predict(dtest)
    preds = np.round(preds)
    print("Accuracy of the default model:", "%.4f " %
          accuracy_score(y_test, preds))

    # Searching for optimal tree depth to prevent overfitting
    # early stopping rounds specidies the number of epochs after which the training will stop if there is no improvement in accuracy
    best_params = grid_train_xgboost(
        X_train, y_train, X_test, y_test, early_stopping_rounds=10, num_boost_round=100)

    print('best_params', best_params)

    # Make predictions with best_params
    print('=======================  best model  ========================')
    evals_result = {}
    bst = xgb.train(best_params, dtrain, num_boost_round=200, early_stopping_rounds=20, evals=[
        (dtest, 'test'), (dtrain, 'train')], verbose_eval=False, evals_result=evals_result)
    # Make predictions and evaluate the best model
    preds = bst.predict(dtest)
    preds = np.round(preds)
    print("Accuracy of the best model:", "%.4f " %
          accuracy_score(y_test, preds))
    plot_eval_metrics(evals_result, title='best model')

    # train model with custom parameters
    print('=======================  custom model  ========================')
    custom_params = {'objective': 'multi:softmax', 'num_class': 4,
                     'max_depth': 10, 'min_child_weight': 5, 'eval_metric': 'merror'}
    evals_result = {}
    model = xgb.train(custom_params, dtrain, num_boost_round=200, early_stopping_rounds=20, evals=[
        (dtest, 'test'), (dtrain, 'train')], verbose_eval=False, evals_result=evals_result)
    # Make predictions and evaluate the custom model
    preds = model.predict(dtest)
    preds = np.round(preds)
    print("Accuracy of the custom model:", "%.4f " %
          accuracy_score(y_test, preds))
    plot_eval_metrics(evals_result, title='custom model')
    plot_feature_importance(model, title='custom model')
    return model, bst


In [ ]:
model, bst = train_xgboost_matrix(colonies_matrix_features, augment=True, )


In [ ]:
# save the model with custom parameters
best_model_path = 'best_model.json'
custom_model_path = 'custom_model.json'
model.save_model(custom_model_path)
bst.save_model(best_model_path)


## Test on unseen images

we test on a colony that haven't been seen by the train set at all.


In [ ]:
colony0_segmentation_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_segmentation_T0_to_T146_trimmed.h5'
colony0_lineage_GT_path = '/home/farzaneh/Documents/Bread/bread/src/bread/tests/data/V2022_09_19_HTB2_mCh_MYO1-GFP_50_ms/FOV0_lineage_T0_to_T146.csv'

colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path)
colony0_lineage_gt.rename(columns={'parent_id': 'parent_GT'}, inplace=True)


In [ ]:

args = {"nn_threshold": 8.0, "flexible_nn_threshold": True,
        "num_frames_refractory": 0, "num_frames": 4, "bud_distance_max": 10.0}
colony0_features = extract_features(colony0_segmentation_path, args)
colony0_features['colony'] = [0 for i in range(len(colony0_features))]
colony0_lineage_gt = pd.read_csv(colony0_lineage_GT_path).rename(
    columns={'parent_id': 'parent_GT'})
colony0_lineage_gt['colony'] = [0 for i in range(len(colony0_lineage_gt))]
colony0_matrix_features = get_matrix_features(
    colony0_features, colony0_lineage_gt)
X = colony0_matrix_features['features'].to_numpy()
y = colony0_matrix_features['parent_index_in_candidates'].to_numpy()

X = flatten_3d_array(X)
dtest = xgb.DMatrix(X, label=y)
saved_model = xgb.Booster()
saved_model.load_model(best_model_path)
saved_model_preds = saved_model.predict(dtest)
saved_model_preds = np.round(saved_model_preds)
print("Accuracy of the best saved model:", "%.4f " %
      accuracy_score(y, saved_model_preds))
print(np.sum(saved_model_preds != y))


In [ ]:
# Check if the accuracy is better when we only use the first 100 cells.
trim_data = colony0_matrix_features.loc[colony0_matrix_features['time_index'] < 50]
X = trim_data['features'].to_numpy()
y = trim_data['parent_index_in_candidates'].to_numpy()

X = flatten_3d_array(X)
dtest = xgb.DMatrix(X, label=y)
saved_model_preds = saved_model.predict(dtest)
saved_model_preds = np.round(saved_model_preds)
print("Accuracy of the saved model for first frames:", "%.4f " %
      accuracy_score(y, saved_model_preds))
print(np.sum(saved_model_preds != y))


the accuracy for 100 first frames is better than for all frames (146) (between 3 to 6 percents difference). The accuracy for 50 first frame is also around the same and it's close to validation prediction in model. we can conclude that the prediction get harder when image is much more crowded.


## Add FOV0 buds to colonies and train on all


In [ ]:
pd.set_option("display.max_rows", 10, "display.max_columns", 20)


In [ ]:
all_matrix_features = colony0_matrix_features.append(
    colonies_matrix_features).reset_index(drop=True)
all_matrix_features


In [ ]:
# train and test on the combined data

model, bst = train_xgboost_matrix(all_matrix_features, augment=True, )
model_path = 'model_all_colonies.json'
model.save_model(model_path)


## train on 4 colonies and test on 2

colony 4 and 5 have chosen for test because 4 was a problematic colony and 5 was a normal one.


In [ ]:
train_matrix_features = all_matrix_features[~ (
    (all_matrix_features['colony'] == 4) | (matrix_feature_df['colony'] == 5))]
test_matrix_features = all_matrix_features[(
    all_matrix_features['colony'] == 4) | (matrix_feature_df['colony'] == 5)]
X_train = train_matrix_features['features'].to_numpy()
X_test = test_matrix_features['features'].to_numpy()
y_train = train_matrix_features['parent_index_in_candidates'].to_numpy()
y_test = test_matrix_features['parent_index_in_candidates'].to_numpy()
# Train the default XGBoost model
dtrain = xgb.DMatrix(X_train, label=y_train)
params = {'objective': 'multi:softmax',
          'num_class': 4, 'eval_metric': 'merror'}
bst = xgb.train(params, dtrain, num_boost_round=100)
# Make predictions and evaluate the model
dtest = xgb.DMatrix(X_test, label=y_test)
preds = bst.predict(dtest)
preds = np.round(preds)
print("Accuracy of the default model:", "%.4f " %

      accuracy_score(y_test, preds))
